## 6 - Attention is All You Need

> Trong notebook lần này, chúng ta cùng nhau implemmenting (và có điều chỉnh một chút) một *Transformer Model* từ paper [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf). Để hiểu thêm về Transformer, nhấn vào 3 đường link sau: [1](https://www.mihaileric.com/posts/transformers-attention-in-disguise/), [2](https://jalammar.github.io/illustrated-transformer/), [3](http://nlp.seas.harvard.edu/2018/04/03/attention.html).

> Dưới đây là kiến trúc Transformer model với Encoder (bên trái) và Decoder (bên phải).

![figure1](./images/6.transformer_model.PNG)

### Introduction

> Tương tự như Convolutional Sequence-to-Sequence model, Transformer model không sử dụng recurrence, nó cũng không sử dụng bất kỳ convolutional layers nào. Thay vào đó, model được xây dựng từ các linear layers, attentions mechanisms và normalization.

> Kể từ tháng Một năm 2020, Transformers trở thành một kiến trúc nổi bật trong NLP, đạt được rất nhiều kết quả state-of-the-art trong nhiều tasks khác nhau. 

> Một trong những biến thể phổ biến nhất của Transformer là [*BERT*](https://arxiv.org/pdf/1810.04805.pdf) (**B**idirectional **E**ncoder **R**epresentantions from **T**ransformers) và phiên bản pre-trained BERT, chúng thường được sử dụng để thay thế embedding layers và làm một số tác vụ khác trong các NLP models.

> Một thư viện phổ biến được sử dụng khi làm việc với pre-trained transformers là [Transformers](https://huggingface.co/docs/transformers/index) library, nhấn vào [đây](https://huggingface.co/models) để biết thêm về các pre-trained models có sẵn.

> Dưới đây là một số khác biệt trong notebook và paper: <br>
>>> * Ở đây, chúng ta sẽ sử dụng các bộ positional encoding động (tức tự học) thay vì các bộ tĩnh.
>>> * Bên cạnh đó, chúng ta cũng sử dụng Adam optimizer với learning rate tĩnh thay vì warm-up và cool-down steps.
>>> * Cuối cùng, chúng ta không áp dụng label smoothing trong notebook này.

> Chúng ta sẽ làm cho những thay đổi này gần với các set-up của BERT và phần lớn các biến thể của Transformer models đều sử dụng các set-up tương tự.

### Preparing the Data

> Import các thư viện và thiết lập random seed.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Load các tokenizers.

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

> Tạo các trường data. Lưu ý `batch_first = True`.

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '', 
            eos_token = '', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '', 
            eos_token = '', 
            lower = True, 
            batch_first = True)

> Load Multi30k dataset và build vocabulary.

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [7]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

> Tạo Iterator và đưa vào device.

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

### Building the Model

> Tương tự như các notebook trước, model lần này cũng được tạo bởi 2 khối: encoder và decoder. Encoder thực hiện mã hóa input/source sentence (Tiếng Đức) thành *context vector* và, decoder thực hiện giải mã *context vector* thành ouput/target sentence (Tiếng Anh).

##### Encoder

> Tương tự như Convolutional Seq2Seq model, encoder của Transformer không nén toàn bộ source sentence, **X = ($x_1$, ..., $x_n$)** vào một context vector duy nhất, z. Thay vào đó, encoder của model sẽ sinh ra một chuỗi các context vectors **Z = ($z_1$, ..., $z_n$)**. Vậy, nếu input sequence có chiều dài là 5 tokens, chúng ta sẽ thu được chuỗi 5 context vectors **Z = ($z_1$, $z_2$, $z_3$, $z_4$, $z_5$)**. <br>
>> **NOTE:** Chúng ta gọi sequence này là một *chuỗi các context vectors* thay vì *chuỗi các hidden state* là do: một hidden state tại thời điểm t trong RNN chỉ quan sát token $x_t$ và tất cả các tokens trước đó. Tuy nhiên, mỗi context vector được sinh ra bởi Transformer model đều có thể quan sát tất cả các tokens cũng vị trí của chúng trong input sequence.

>>>> ![figure2](./images/6.encoder.PNG)

> Đầu tiên, các tokens được truyền vào một embedding layer. Tiếp theo, do model không sử dụng recurrent nên nó không thể nhận biết được thứ tự của các tokens bên trong sequence. Ta giải quyết vấn đề này bằng cách sử dụng embedding layer thứ 2, được gọi là *positional embedding layer*. Lớp positional embedding này nhận đầu vào không phải là token mà là vị trí của token trong sequence, bắt đầu với token đầu tiên \<sos> token, ở vị trí 0. Position embedding có "vocabulary" với kích thước bằng 100, có nghĩa chúng ta có thể xử lý sequence có độ dài lên tới 100 tokens. Ta hoàn toàn có thể thay đổi tham số này nếu muốn xử lý sequence dài hơn.

> Phiên bản gốc của Transformer từ paper Attention is All You Need không học positional embedding. Thay vào đó, nó sử dụng embedding có định. Các kiến trúc Transformer hiện đại như BERT sử dụng positional embedding và đạt hiệu suất tốt hơn. Nhấn vào [đây](http://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding) để đọc thêm về positional embedding được sử dụng trong mô hình Transformer gốc.

> Tiếp theo, token và positional embeddings được *elementwise summed* với nhau để sinh ra vector chứa thông tin về token cũng như vị trí của chúng bên trong sequence. Tuy nhiên, truocs khi được cộng, token embedding được nhân với hệ số tỷ lệ $\sqrt{d_{model}}$, với $d_{model}$ là hidden dimension size `hid_dim`. Ta thực hiện công việc scaling trên để giảm phương sai trong embedding. Nếu không có hệ số nhân này, model có vẻ không đáng tin cậy. Sau đó,ta áp dụng Dropout sao khi cộng hai vector token embeddings và positional embeddings.

> Embedding cùng được truyền qua N *encoder layers* để thu được Z, làm đầu vào cho decoder.

> Source mask, `src_mask` có kích thước bằng với source sentence nhưng có các phần tử chỉ bằng 0 hoặc 1 (1 khi token trong source sentence không phải là padding token và bằng 0 khi nó là padding token). Nó được sử dụng trong encoder layers để che đi multi-head attention mechanisms - cơ chế này được tính toán và áp dụng attention trên source sentence, do đó model không cần thiết phải chú ý đến padding token vì chúng không mang thông tin hữu ích.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

##### Encoder Layer

> Trong encoder layer, đầu tiên chúng ta truyền source sentence và mask của nó vào *multi-head attention layer*, thực hiện dropout, áp dụng residual connection và truyền chúng qua [Layer Normalization](https://arxiv.org/pdf/1607.06450.pdf) layer. Sau đó chúng ta truyền nó qua một *position-wise feedforward* layer và một lần nữa, áp dụng dropout, một residual connection và sau cùng normalization để thu được đầu ra của encoder layer này và đưa vào encoder layer tiếp theo. Lưu ý là parameters không được chia sẻ giữa các layers.

> Multi head attention layer được sử dụng bởi encoder layer để tập trung chú ý vào source sentence, nó tính toán và áp dụng attention lên chính nó thay vì các sequence khác nên được gọi là *self-attention*.

> Về cơ bản, layer normalization là thực hiện chuẩn hóa giá trị thuộc tính, ví dụ theo hidden dimension, do đó mỗi thuộc tính có mean bằng 0, độ lệch chuẩn bằng 1. Việc thực hiện layer normalization giúp neural networks với nhiều layers như Transformer được train dễ dàng hơn.

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

##### Multi Head Attention Layer

> Một trong những ý tưởng hay, quan trọng nhất trong Transformer chính là *multi-head attention layer*.

![figure3](./images/6.attention.PNG)